In [59]:
from pathlib import Path
import pandas as pd
import numpy as np
import json

In [60]:
pd.set_option('display.max_columns', 1000)

In [83]:
# Defining directories
dir_path = Path.cwd().parent
prep_folder = dir_path.joinpath("data/prep")
df_path = prep_folder.joinpath("merged.parquet")
model_df_path = dir_path.joinpath("data/model_df.csv")
species_dict_path = dir_path.joinpath("data/species_dict.json")


In [62]:
df = pd.read_parquet(df_path)
index_cols = ["long", "lat", "season", "species_names"]
df

,long,lat,year,month,species_names,vh_Ascending,vv_Ascending,vh_Descending,vv_Descending,B02,B03,B04,B08,B11,B12
0,75.66669580000007,32.25107380000003,2018,01,bambusa bamboo,95.0,174.0,80.0,208.0,NaN,NaN,NaN,NaN,NaN,NaN
1,75.66669580000007,32.25107380000003,2018,02,bambusa bamboo,142.5,165.5,99.0,211.0,1408.0,1612.0,1683.0,2892.0,2624.0,2073.0
2,75.66669580000007,32.25107380000003,2018,03,bambusa bamboo,103.5,234.0,74.0,137.0,1390.0,1710.0,1764.0,3186.0,2831.0,2246.0
3,75.66669580000007,32.25107380000003,2018,04,bambusa bamboo,99.0,218.0,97.0,180.0,1544.0,1772.0,1755.0,3418.0,2897.0,2356.0
4,75.66669580000007,32.25107380000003,2018,05,bambusa bamboo,99.0,194.0,99.5,196.0,2160.0,2360.0,2360.0,3242.0,3259.0,2677.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12129091,78.12624330000006,31.258713300000068,2024,08,rhododendron,142.0,161.5,92.5,204.0,1168.0,1270.0,1190.0,3315.0,2159.0,1617.0
12129092,78.12624330000006,31.258713300000068,2024,09,rhododendron,129.0,318.0,80.0,89.0,1147.0,1426.0,1124.0,3230.5,1956.5,1472.0
12129093,78.12624330000006,31.258713300000068,2024,10,rhododendron,119.0,191.5,59.5,65.5,999.5,1071.0,1047.0,2238.5,1817.5,1289.5
12129094,78.12624330000006,31.258713300000068,2024,11,rhododendron,86.0,165.0,47.0,78.0,1269.5,1363.5,1361.5,2179.5,1819.5,1490.0


In [63]:
# creating seasons
conds = [df["month"].isin(["03", "04", "05"]), df["month"].isin(["10", "11", "12"])]
opts = ["summer", "winter"]
df["season"] = np.select(conds, opts, default="NA")
df = df[~(df["season"] == "NA")]
df = df.drop(["year", "month"], axis=1)

df

,long,lat,species_names,vh_Ascending,vv_Ascending,vh_Descending,vv_Descending,B02,B03,B04,B08,B11,B12,season
2,75.66669580000007,32.25107380000003,bambusa bamboo,103.5,234.0,74.0,137.0,1390.0,1710.0,1764.0,3186.0,2831.0,2246.0,summer
3,75.66669580000007,32.25107380000003,bambusa bamboo,99.0,218.0,97.0,180.0,1544.0,1772.0,1755.0,3418.0,2897.0,2356.0,summer
4,75.66669580000007,32.25107380000003,bambusa bamboo,99.0,194.0,99.5,196.0,2160.0,2360.0,2360.0,3242.0,3259.0,2677.0,summer
9,75.66669580000007,32.25107380000003,bambusa bamboo,106.0,271.5,141.0,233.0,1351.0,1582.0,1493.0,3620.0,2536.0,1841.0,winter
10,75.66669580000007,32.25107380000003,bambusa bamboo,132.0,188.0,111.0,259.5,1363.0,1540.0,1458.0,3499.0,2460.0,1786.0,winter
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12129087,78.12624330000006,31.258713300000068,rhododendron,107.5,131.5,53.0,80.0,1867.5,1901.0,1707.0,3052.5,2211.5,1742.5,summer
12129088,78.12624330000006,31.258713300000068,rhododendron,144.0,269.0,70.0,152.0,1287.0,1549.5,1342.5,4169.0,2934.0,2048.0,summer
12129093,78.12624330000006,31.258713300000068,rhododendron,119.0,191.5,59.5,65.5,999.5,1071.0,1047.0,2238.5,1817.5,1289.5,winter
12129094,78.12624330000006,31.258713300000068,rhododendron,86.0,165.0,47.0,78.0,1269.5,1363.5,1361.5,2179.5,1819.5,1490.0,winter


In [64]:
# Making indices
def make_indices(df):

    #### SENTINEL 2

    ## NDVI
    df["NDVI"] = (df["B08"] - df["B04"]) / (df["B08"] + df["B04"])
    temp_df = pd.DataFrame(df["NDVI"])

    ## Atmospherically Resistant Vegetation Index 2
    df["ARVI2"] = -0.18 + 1.17 * df["NDVI"]

    ## Blue-wide dynamic range vegetation index
    df["BWDRVI"] = (0.1 * (df["B08"] - df["B02"])) / (0.1 * (df["B08"] + df["B02"]))

    ## Chlorophyll vegetation index
    df["CVI"] = df["B08"] * (df["B04"] / (df["B03"].pow(2)))

    ## Corrected Transformed Vegetation Index
    temp_df["ndvi+0.5"] = temp_df["NDVI"] + 0.5
    temp_df["ctvi1"] = (temp_df["ndvi+0.5"]) / (temp_df["ndvi+0.5"].abs())
    temp_df["ctvi2"] = np.sqrt(temp_df["ndvi+0.5"].abs())
    df["CTVI"] = temp_df["ctvi1"] * temp_df["ctvi2"]

    ## Enhanced Vegetation Index 2 -2
    df["EVI2"] = 2.5 * ((df["B08"] - df["B04"]) / (df["B08"] + (2.4 * df["B04"]) + 1))

    ## Global Vegetation Moisture Index
    temp_df['gvmi_num'] = (df['B08'] + 0.1) - (df['B11'] + 0.02)
    temp_df['gvmi_den'] = (df['B08'] + 0.1) + (df['B11'] + 0.02)
    df['GVMI'] = temp_df['gvmi_num']/temp_df['gvmi_den']

    ## Mid-infrared vegetation index
    df['MVI'] = df['B08']/df['B11']

    ## Modified Soil Adjusted Vegetation Index hyper
    temp_df['2800+1'] = df['B12'] + 1
    temp_df['hyper_second'] = np.sqrt(temp_df['2800+1'].pow(2) - (8 * (df['B08'] - df['B04'])))
    df['MSVAIhyper'] = 0.5 * (temp_df['2800+1'] - temp_df['hyper_second'])

    ## Modified Triangular Vegetation Index 2
    # temp_df['mtvi2_num'] = 1.2 * (df['B08'] - df['B03']) - 2.5 *(df['B04'] - df['B03'])
    # df['MTVI2']


    ## Normalized Difference NIR/MIR Modified Normalized Difference Vegetation Index
    df['MNDVI'] = (df['B08'] - df['B03'])/(df['B08'] + df['B03'])

    ## Optimized Soil Adjusted Vegetation Index
    temp_df['Y'] = 0.16
    temp_df['osavi_num'] = df['B08'] - df['B04']
    temp_df['osavi_den'] = df['B08'] + df['B04'] + temp_df['Y']
    df['OSAVI'] = (temp_df['Y'] + 1) * (temp_df['osavi_num']/temp_df['osavi_den'])

    ## Perpendicular Vegetation Index
    # df['PVI'] = 

    ## Soil and Atmospherically Resistant Vegetation Index
    # df['SARVI]

    ## Specific Leaf Area Vegetation Index
    df['SLAVI'] = df['B08'] / (df['B04'] + df['B11'])

    ## Transformed Soil Adjusted Vegetation Index 2

    ## Weighted Difference Vegetation Index

    ## Wide Dynamic Range Vegetation Index
    df['WDRVI'] = 0.1*(df['B08'] - df['B04'])/(0.1*(df['B08'] + df['B04']))



    #### SENTINEL 
    ## VV VH Ratio
    df['VV_VH_A'] = df['vv_Ascending'] / df['vh_Ascending']
    df['VV_VH_D'] = df['vv_Descending'] / df['vh_Descending']

    ## VH VV Ratio
    df['VH_VV_A'] = df['vh_Ascending'] / df['vv_Ascending']
    df['VH_VV_D'] = df['vh_Descending'] / df['vv_Descending']

    ## SAR Normalized Difference Vegetation Index 
    df['SAR_NDVI_A'] = (df['vh_Ascending'] - df['vv_Ascending']) / (df['vh_Ascending'] + df['vv_Ascending'])
    df['SAR_NDVI_D'] = (df['vh_Descending'] - df['vv_Descending']) / (df['vh_Descending'] + df['vv_Descending'])

    ## Difference Vegetation Index
    df['DVI_A'] = df['vh_Ascending'] - df['vv_Ascending']
    df['DVI_D'] = df['vh_Descending'] - df['vv_Descending']

    ## Sum Vegetation Index
    df['SVI_A'] = df['vh_Ascending'] + df['vv_Ascending']
    df['SVI_D'] = df['vh_Descending'] + df['vv_Descending']

    ## Relative Difference Vegetation Index (RDVI)
    df['RDVI_A'] = df['VH_VV_A'] - df['VV_VH_A']
    df['RDVI_D'] = df['VH_VV_D'] - df['VV_VH_D']

    ## Normalized Relative Difference Vegetation Index
    df['NRDVI_A'] = df['RDVI_A'] / (df['VH_VV_A'] + df['VV_VH_A'])
    df['NRDVI_D'] = df['RDVI_D'] / (df['VH_VV_D'] + df['VV_VH_D'])

    ## Sum of Squared Difference Vegetation Index
    df['SSDVI_A'] = df['vh_Ascending'].pow(2) - df['vv_Ascending'].pow(2)
    df['SSDVI_D'] = df['vh_Descending'].pow(2) - df['vv_Descending'].pow(2)

    return df, temp_df

df_indices, temp_DF = make_indices(df)




In [65]:
df_indices

,long,lat,species_names,vh_Ascending,vv_Ascending,vh_Descending,vv_Descending,B02,B03,B04,B08,B11,B12,season,NDVI,ARVI2,BWDRVI,CVI,CTVI,EVI2,GVMI,MVI,MSVAIhyper,MNDVI,OSAVI,SLAVI,WDRVI,VV_VH_A,VV_VH_D,VH_VV_A,VH_VV_D,SAR_NDVI_A,SAR_NDVI_D,DVI_A,DVI_D,SVI_A,SVI_D,RDVI_A,RDVI_D,NRDVI_A,NRDVI_D,SSDVI_A,SSDVI_D
2,75.66669580000007,32.25107380000003,bambusa bamboo,103.5,234.0,74.0,137.0,1390.0,1710.0,1764.0,3186.0,2831.0,2246.0,summer,0.287273,0.156109,0.392483,1.921994,0.887284,0.479072,0.059012,1.125397,1.266401,0.301471,0.333226,0.693362,0.287273,2.260870,1.851351,0.442308,0.540146,-0.386667,-0.298578,-130.5,-63.0,337.5,211.0,-1.818562,-1.311205,-0.672750,-0.548278,-44043.75,-13293.00
3,75.66669580000007,32.25107380000003,bambusa bamboo,99.0,218.0,97.0,180.0,1544.0,1772.0,1755.0,3418.0,2897.0,2356.0,summer,0.321477,0.196128,0.377670,1.910389,0.906354,0.544817,0.082513,1.179841,1.411962,0.317148,0.372902,0.734738,0.321477,2.202020,1.855670,0.454128,0.538889,-0.375394,-0.299639,-119.0,-83.0,317.0,277.0,-1.747892,-1.316781,-0.658055,-0.549906,-37723.00,-22991.00
4,75.66669580000007,32.25107380000003,bambusa bamboo,99.0,194.0,99.5,196.0,2160.0,2360.0,2360.0,3242.0,3259.0,2677.0,summer,0.157444,0.004209,0.200296,1.373729,0.810829,0.247558,-0.002603,0.994784,0.658863,0.157444,0.182630,0.576971,0.157444,1.959596,1.969849,0.510309,0.507653,-0.324232,-0.326565,-95.0,-96.5,293.0,295.5,-1.449287,-1.462196,-0.586778,-0.590190,-27835.00,-28515.75
9,75.66669580000007,32.25107380000003,bambusa bamboo,106.0,271.5,141.0,233.0,1351.0,1582.0,1493.0,3620.0,2536.0,1841.0,winter,0.415998,0.306718,0.456447,2.159511,0.957078,0.738111,0.176098,1.427445,2.312349,0.391772,0.482543,0.898486,0.415998,2.561321,1.652482,0.390424,0.605150,-0.438411,-0.245989,-165.5,-92.0,377.5,374.0,-2.170897,-1.047332,-0.735462,-0.463907,-62476.25,-34408.00
10,75.66669580000007,32.25107380000003,bambusa bamboo,132.0,188.0,111.0,259.5,1363.0,1540.0,1458.0,3499.0,2460.0,1786.0,winter,0.411741,0.301737,0.439325,2.151097,0.954851,0.729012,0.174368,1.422358,2.287203,0.388768,0.477604,0.893058,0.411741,1.424242,2.337838,0.702128,0.427746,-0.175000,-0.400810,-56.0,-148.5,320.0,370.5,-0.722115,-1.910092,-0.339600,-0.690665,-17920.00,-55019.25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12129087,78.12624330000006,31.258713300000068,rhododendron,107.5,131.5,53.0,80.0,1867.5,1901.0,1707.0,3052.5,2211.5,1742.5,summer,0.282698,0.150756,0.240854,1.441866,0.884702,0.470435,0.159776,1.380285,1.544816,0.232462,0.327918,0.778997,0.282698,1.223256,1.509434,0.817490,0.662500,-0.100418,-0.203008,-24.0,-27.0,239.0,133.0,-0.405765,-0.846934,-0.198832,-0.389945,-5736.00,-3591.00
12129088,78.12624330000006,31.258713300000068,rhododendron,144.0,269.0,70.0,152.0,1287.0,1549.5,1342.5,4169.0,2934.0,2048.0,summer,0.512837,0.420019,0.528226,2.331111,1.006398,0.955932,0.173879,1.420927,2.762632,0.458075,0.594873,0.974863,0.512837,1.868056,2.171429,0.535316,0.460526,-0.302663,-0.369369,-125.0,-82.0,413.0,222.0,-1.332740,-1.710902,-0.554529,-0.650050,-51625.00,-18204.00
12129093,78.12624330000006,31.258713300000068,rhododendron,119.0,191.5,59.5,65.5,999.5,1071.0,1047.0,2238.5,1817.5,1289.5,winter,0.362654,0.244305,0.382644,2.043266,0.928792,0.626802,0.103813,1.231637,1.849221,0.352772,0.420658,0.781463,0.362654,1.609244,1.100840,0.621410,0.908397,-0.233494,-0.048000,-72.5,-6.0,310.5,125.0,-0.987834,-0.192443,-0.442845,-0.095779,-22511.25,-750.00
12129094,78.12624330000006,31.258713300000068,rhododendron,86.0,165.0,47.0,78.0,1269.5,1363.5,1361.5,2179.5,1819.5,1490.0,winter,0.231008,0.090280,0.263845,1.596115,0.854990,0.375360,0.090040,1.197857,1.098059,0.230313,0.267957,0.685162,0.231008,1.918605,1.659574,0.521212,0.602564,-0.314741,-0.248000,-79.0,-31.0,251.0,125.0,-1.397393,-1.057010,-0.572745,-0.467262,-19829.00,-3875.00


In [66]:
#Grouping vars by season
df_indices = df_indices.groupby(index_cols).median().reset_index()

# check for missingness
print(df_indices.shape)
df_indices.dropna(axis=0, how='any')



(288788, 43)


,long,lat,season,species_names,vh_Ascending,vv_Ascending,vh_Descending,vv_Descending,B02,B03,B04,B08,B11,B12,NDVI,ARVI2,BWDRVI,CVI,CTVI,EVI2,GVMI,MVI,MSVAIhyper,MNDVI,OSAVI,SLAVI,WDRVI,VV_VH_A,VV_VH_D,VH_VV_A,VH_VV_D,SAR_NDVI_A,SAR_NDVI_D,DVI_A,DVI_D,SVI_A,SVI_D,RDVI_A,RDVI_D,NRDVI_A,NRDVI_D,SSDVI_A,SSDVI_D
0,75.66669580000007,32.25107380000003,summer,bambusa bamboo,107.0,187.0,101.5,212.5,1440.00,1712.00,1755.0,3259.00,2908.0,2314.00,0.313389,0.186665,0.384061,1.979580,0.901881,0.529084,0.059619,1.126771,1.410168,0.317148,0.363521,0.720576,0.313389,1.802632,1.969849,0.554745,0.507653,-0.286385,-0.326565,-88.0,-102.5,301.0,330.0,-1.247887,-1.462196,-0.529354,-0.590190,-23838.75,-36131.25
1,75.66669580000007,32.25107380000003,winter,bambusa bamboo,118.5,242.0,102.0,212.0,1380.50,1581.00,1534.0,3534.00,2506.5,1832.50,0.416930,0.307808,0.437030,2.175362,0.957564,0.740110,0.186492,1.458862,2.375948,0.391200,0.483624,0.913598,0.416930,2.027778,2.030534,0.493151,0.492481,-0.339450,-0.340050,-116.5,-92.0,345.0,310.0,-1.534627,-1.538053,-0.608755,-0.609609,-43340.00,-34408.00
2,75.66676390000003,32.25073910000003,summer,bambusa bamboo,57.0,115.0,85.0,188.0,1384.00,1654.00,1670.0,3382.00,3206.0,2529.00,0.345016,0.223668,0.421399,2.062734,0.919247,0.591313,0.034362,1.071142,1.342598,0.353979,0.400206,0.691076,0.345016,2.128205,2.361111,0.469880,0.423529,-0.360656,-0.404959,-63.0,-98.0,164.0,287.0,-1.658326,-1.937582,-0.638288,-0.695810,-10521.00,-27071.00
3,75.66676390000003,32.25073910000003,winter,bambusa bamboo,63.5,140.5,96.0,194.0,1337.00,1537.00,1436.0,3463.50,2717.5,1955.00,0.421175,0.312774,0.430745,2.144670,0.959778,0.749224,0.126209,1.288905,2.127773,0.396278,0.488547,0.848308,0.421175,1.873239,2.067797,0.533835,0.483607,-0.303922,-0.348066,-62.0,-93.0,223.5,294.0,-1.339405,-1.584190,-0.556445,-0.620909,-13298.25,-27927.00
4,75.66686810000004,32.25026770000005,summer,bambusa bamboo,103.0,164.5,130.0,241.0,1455.00,1737.00,1784.0,3386.00,3433.0,2720.00,0.317657,0.191659,0.414938,2.029689,0.904244,0.537360,0.019891,1.040565,1.253600,0.326590,0.368470,0.677656,0.317657,1.610329,1.905797,0.620991,0.524715,-0.233813,-0.311721,-66.0,-102.0,266.0,380.0,-0.989337,-1.381082,-0.443387,-0.568227,-18445.00,-38760.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
288783,78.12614830000007,31.258660000000077,winter,rhododendron,123.0,202.0,70.0,129.0,1289.50,1252.00,1170.0,1993.00,1618.0,1318.50,0.236928,0.097206,0.200712,1.513556,0.858445,0.386012,0.095046,1.210016,1.087045,0.233678,0.274823,0.684173,0.236928,1.623583,1.794393,0.615922,0.557292,-0.237684,-0.284281,-83.0,-58.0,322.0,217.0,-1.007661,-1.237101,-0.449948,-0.526049,-24420.00,-10672.00
288784,78.12618260000005,31.25882800000005,summer,rhododendron,118.0,215.5,69.0,104.0,1401.25,1586.75,1334.0,2932.75,2350.0,1771.75,0.318470,0.192610,0.311673,1.740771,0.904683,0.538985,0.122675,1.280013,1.465306,0.300679,0.369410,0.741655,0.318470,1.722656,1.574257,0.580499,0.635220,-0.265423,-0.223077,-115.5,-35.0,326.5,170.0,-1.142157,-0.939037,-0.495910,-0.425004,-35264.00,-6240.00
288785,78.12618260000005,31.25882800000005,winter,rhododendron,108.0,178.0,78.0,136.0,1263.50,1319.50,1217.0,2250.50,1836.0,1432.50,0.277768,0.144988,0.268091,1.671398,0.881911,0.461126,0.097416,1.215822,1.326172,0.272531,0.322192,0.699944,0.277768,1.694253,1.895028,0.590231,0.527697,-0.257679,-0.309160,-76.0,-68.0,282.5,217.0,-1.104022,-1.367331,-0.483270,-0.564377,-20020.00,-12172.00
288786,78.12624330000006,31.258713300000068,summer,rhododendron,141.0,243.0,66.0,105.0,1429.50,1615.75,1482.5,3262.75,2295.0,1815.75,0.364454,0.246412,0.376572,1.897861,0.929756,0.630527,0.164205,1.392910,1.829137,0.337570,0.422752,0.828370,0.364454,1.844221,1.600000,0.542234,0.625000,-0.296820,-0.230769,-131.0,-35.0,366.0,175.0,-1.301987,-0.975000,-0.545573,-0.438202,-44183.25,-6963.00


In [67]:
# widening the frame
df_wide = df_indices.pivot(index=['long', 'lat', 'species_names'], columns=['season']).reset_index()
new_cols = ["".join(x) for x in df_wide.columns]
df_wide.columns = new_cols
df_wide

,long,lat,species_names,vh_Ascendingsummer,vh_Ascendingwinter,vv_Ascendingsummer,vv_Ascendingwinter,vh_Descendingsummer,vh_Descendingwinter,vv_Descendingsummer,vv_Descendingwinter,B02summer,B02winter,B03summer,B03winter,B04summer,B04winter,B08summer,B08winter,B11summer,B11winter,B12summer,B12winter,NDVIsummer,NDVIwinter,ARVI2summer,ARVI2winter,BWDRVIsummer,BWDRVIwinter,CVIsummer,CVIwinter,CTVIsummer,CTVIwinter,EVI2summer,EVI2winter,GVMIsummer,GVMIwinter,MVIsummer,MVIwinter,MSVAIhypersummer,MSVAIhyperwinter,MNDVIsummer,MNDVIwinter,OSAVIsummer,OSAVIwinter,SLAVIsummer,SLAVIwinter,WDRVIsummer,WDRVIwinter,VV_VH_Asummer,VV_VH_Awinter,VV_VH_Dsummer,VV_VH_Dwinter,VH_VV_Asummer,VH_VV_Awinter,VH_VV_Dsummer,VH_VV_Dwinter,SAR_NDVI_Asummer,SAR_NDVI_Awinter,SAR_NDVI_Dsummer,SAR_NDVI_Dwinter,DVI_Asummer,DVI_Awinter,DVI_Dsummer,DVI_Dwinter,SVI_Asummer,SVI_Awinter,SVI_Dsummer,SVI_Dwinter,RDVI_Asummer,RDVI_Awinter,RDVI_Dsummer,RDVI_Dwinter,NRDVI_Asummer,NRDVI_Awinter,NRDVI_Dsummer,NRDVI_Dwinter,SSDVI_Asummer,SSDVI_Awinter,SSDVI_Dsummer,SSDVI_Dwinter
0,75.66669580000007,32.25107380000003,bambusa bamboo,107.0,118.5,187.0,242.0,101.5,102.0,212.5,212.0,1440.00,1380.5,1712.00,1581.0,1755.00,1534.0,3259.00,3534.0,2908.00,2506.5,2314.00,1832.5,0.313389,0.416930,0.186665,0.307808,0.384061,0.437030,1.979580,2.175362,0.901881,0.957564,0.529084,0.740110,0.059619,0.186492,1.126771,1.458862,1.410168,2.375948,0.317148,0.391200,0.363521,0.483624,0.720576,0.913598,0.313389,0.416930,1.802632,2.027778,1.969849,2.030534,0.554745,0.493151,0.507653,0.492481,-0.286385,-0.339450,-0.326565,-0.340050,-88.0,-116.5,-102.5,-92.0,301.0,345.0,330.0,310.0,-1.247887,-1.534627,-1.462196,-1.538053,-0.529354,-0.608755,-0.590190,-0.609609,-23838.75,-43340.00,-36131.25,-34408.00
1,75.66676390000003,32.25073910000003,bambusa bamboo,57.0,63.5,115.0,140.5,85.0,96.0,188.0,194.0,1384.00,1337.0,1654.00,1537.0,1670.00,1436.0,3382.00,3463.5,3206.00,2717.5,2529.00,1955.0,0.345016,0.421175,0.223668,0.312774,0.421399,0.430745,2.062734,2.144670,0.919247,0.959778,0.591313,0.749224,0.034362,0.126209,1.071142,1.288905,1.342598,2.127773,0.353979,0.396278,0.400206,0.488547,0.691076,0.848308,0.345016,0.421175,2.128205,1.873239,2.361111,2.067797,0.469880,0.533835,0.423529,0.483607,-0.360656,-0.303922,-0.404959,-0.348066,-63.0,-62.0,-98.0,-93.0,164.0,223.5,287.0,294.0,-1.658326,-1.339405,-1.937582,-1.584190,-0.638288,-0.556445,-0.695810,-0.620909,-10521.00,-13298.25,-27071.00,-27927.00
2,75.66686810000004,32.25026770000005,bambusa bamboo,103.0,101.5,164.5,188.0,130.0,130.0,241.0,241.0,1455.00,1364.5,1737.00,1617.0,1784.00,1512.5,3386.00,3612.0,3433.00,2849.5,2720.00,2022.0,0.317657,0.402273,0.191659,0.290659,0.414938,0.448741,2.029689,2.164264,0.904244,0.949880,0.537360,0.708932,0.019891,0.129041,1.040565,1.296293,1.253600,2.075946,0.326590,0.390225,0.368470,0.466622,0.677656,0.837366,0.317657,0.402273,1.610329,1.784810,1.905797,1.850394,0.620991,0.560284,0.524715,0.540426,-0.233813,-0.281818,-0.311721,-0.298343,-66.0,-79.5,-102.0,-108.0,266.0,289.0,380.0,376.5,-0.989337,-1.224526,-1.381082,-1.309968,-0.443387,-0.522165,-0.568227,-0.547916,-18445.00,-23580.00,-38760.00,-39096.00
3,75.66719240000003,32.249825100000066,bambusa bamboo,128.0,108.5,209.0,218.5,125.0,124.0,202.0,222.0,1498.00,1357.0,1730.00,1573.5,1798.00,1493.5,3432.00,3561.5,3274.00,2733.0,2559.00,1917.0,0.314274,0.422492,0.187701,0.314315,0.397944,0.449446,1.984213,2.136440,0.902371,0.960429,0.530791,0.752337,0.012687,0.147660,1.025678,1.346985,1.268718,2.416724,0.328808,0.401471,0.364546,0.490077,0.675342,0.897929,0.314274,0.422492,1.576000,1.966443,1.637681,2.063197,0.634518,0.508532,0.610619,0.484685,-0.223602,-0.325792,-0.241758,-0.347087,-75.0,-108.0,-86.0,-117.0,339.0,323.5,335.0,347.0,-0.941482,-1.457911,-1.027062,-1.578512,-0.425910,-0.589061,-0.456817,-0.619539,-27448.00,-31992.00,-24510.00,-35990.00
4,75.66729090000007,32.24962370000003,bambusa bamboo,118.0,107.0,217.0,200.5,116.0,117.0,196.5,220.5,1391.00,1301.0,1646.00,1503.

In [68]:
if not species_dict_path.exists():

    #exporting species dictionary as json
    cats = pd.Categorical(df_wide['species_names'])
    species_dict = dict(zip(range(len(cats.categories)), cats.categories))

    with open(species_dict_path, 'w') as json_file:
        json.dump(species_dict, json_file, indent=4)

In [81]:
df_wide["species"] = pd.Categorical(df_wide['species_names']).codes
df_wide.drop('species_names', axis=1, inplace=True)

In [85]:
df_wide.to_csv(model_df_path, index=False)